In [4]:
import pandas as pd
import numpy as np
import json
import tensorflow as tf
import pickle

from gensim.models import word2vec
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)



In [ ]:
transcript = ('/Users/pranavbarot/Desktop/DataSci/Pranav%2FAll_tasks.json')

In [ ]:
transcript_df = pd.read_json(transcript, lines=True, encoding='utf8')

In [ ]:
transcript_df

In [ ]:
account_file = ('/Users/pranavbarot/Desktop/DataSci/Find_Accounts.json')

In [ ]:
account_df = pd.read_json(account_file, lines=True, encoding='utf8')

In [ ]:
task_list = []
for task in account_df['Task_ID']:
    task_list.append(task)

In [ ]:
for task in task_list:
    print str(task)

In [ ]:
grouped = transcript_df.groupby("Task_AccountId")

In [ ]:
second_dates = []
first_dates = []
call_ids = []
for num, task in enumerate(transcript_df['Task_ID']):
    if task in task_list:
        follow_up =  transcript_df['Task_ID'][num+1]
        follow_up_time = transcript_df['Task_LastModifiedDate'][num+1]
        first_time = transcript_df['Task_LastModifiedDate'][num]
        
        
        second_dates.append(follow_up_time)
        first_dates.append(first_time)
        
        
second_dates        
            

In [ ]:
len(second_dates)

In [ ]:
prediction_df = pd.DataFrame(list(zip(task_list, second_dates, first_dates)), columns = ['Task_ID', 'Follow up Time', 'Call Timestamp'])
prediction_df

In [ ]:
join_file = ('/Users/pranavbarot/Desktop/DataSci/Pranav%2FTasks_and_call_ids.json')
join_df = pd.read_json(join_file, lines=True, encoding = 'utf8')
join_df

In [ ]:
joined = prediction_df.set_index('Task_ID').join(join_df.set_index('task_id'), how='inner')

In [ ]:
joined.reset_index(inplace = True)

In [ ]:
joined

In [ ]:
joined.rename(columns={'index':'Task_ID'}, inplace=True)

In [ ]:
joined_task_ids = set(joined['Task_ID'])

In [ ]:
joined_task_ids

In [ ]:
pred_task_ids = set(prediction_df['Task_ID'])
pred_task_ids

In [ ]:
set(pred_task_ids) == set(joined_task_ids)

In [ ]:
joined_taskcall_only = joined[['Task_ID', 'Follow up Time', 'call_id']]
joined_taskcall_only

In [ ]:
bad_words = []

def filter_callside(transcript):
    """
    Filters the call side to filter our Manager Calls
    when loading the JSON dictionaries in the 'make_features' function

    Args:
        Param1 (dict) : Transcript Dictionary

    Returns:
        Bool (True if the call_side is either caller or callee)

    """
    return (transcript['call_side'] == 'caller' or transcript['call_side'] == 'callee')


def taste_to_text(taste):
    """
    converts the taste fiels from Transcripts into a string by joining them

    """
    return ' '.join([t['raw'] for t in taste if t['raw'] not in bad_words])


def make_features(filename, filter_callside):
    """"
    This functions takes in the JSON file and the filter callside function
    and returns a list of dictionaries with Text of the conversations
    along with dial_id

    """
    transcripts = []
    with open(filename, 'r') as f:

        for line in f.readlines():
            transcript = json.loads(line)

            if filter_callside(transcript):
                transcript['text'] = taste_to_text(transcript['taste'])
                del transcript['taste']

                transcripts.append(transcript)

    return transcripts



In [ ]:
number_of_files = 10
f = '/Users/pranavbarot/Desktop/zenreach_transcripts/Pranav%2Ftranscripts_zenreach%2Fzenreach_transcripts_{}.json'
filenames = [f.format(i) for i in range(number_of_files)]

conv_features = []
  
for files in filenames:

    conv_features.extend(make_features(files, filter_callside))
    print ("File Processed: ", files)

features_df = pd.DataFrame(conv_features)

In [ ]:
cols = ['call_id','call_side']


In [ ]:
grouped_features_df = features_df[['call_id','call_side','created','duration','text']].groupby(cols).max().reset_index(drop = False)
grouped_features_df

In [ ]:
matching_features_df = grouped_features_df.set_index('call_id').join(joined_taskcall_only.set_index('call_id'), how='inner').reset_index(drop = False)

matching_features_df

In [ ]:
position_values = pd.read_csv('/Users/pranavbarot/Desktop/DataSci/position_values.csv')
position_values

In [ ]:
features_with_position = matching_features_df.set_index('call_id').join(position_values.set_index('call_id'), how = 'inner').reset_index(drop=False)
words = features_with_position['words']

In [ ]:
features_with_position= features_with_position[['call_id','call_side','text','created','Follow up Time','position_start','position_end']]
features_with_position['words']=words
features_with_position


In [ ]:
features_with_position['text_list'] = features_with_position['text'].apply(lambda x: x.split())

In [ ]:
features_with_position

In [ ]:
x = features_with_position[features_with_position['call_side'] == 'caller'].reset_index(drop=False)
x

In [ ]:
def get_text(row):
    return row['text_list'][row['position_start']-4:row['position_end']+4]

In [ ]:
def text_window(dataframe):
    new_text = dataframe.apply(lambda row: get_text(row), axis=1)
    return new_text

In [ ]:
text_window(x)

In [ ]:
x['text window'] = text_window(x)
x

In [ ]:
final_training_df = x[['call_id', 'text window', 'Follow up Time']]
final_training_df

In [ ]:
final_training_df.to_pickle('/Users/pranavbarot/Desktop/DataSci/cnn_training_data.p')

In [ ]:
text = final_training_df['text window']
text

In [3]:
with open('/Users/pranavbarot/Desktop/DataSci/cnn_training_data.p') as f:
     final_df = pickle.load(f)
final_df

,call_id,text window,Follow up Time
0,0001e4e4-f332-1235-06b0-0cc47a392728,"[a, call, uh, next, thursday, around, the, sam...",2017-08-10 21:06:22 UTC
1,00062059-e5a4-1235-06b0-0cc47a392728,"[just, text, this, number, monday, morning, an...",2017-07-17 15:03:56 UTC
2,000a56a6-c63c-1235-06b0-0cc47a392728,"[[noise], [noise], [noise], [noise], six, o'cl...",2017-06-08 15:26:20 UTC
3,0014f782-fec4-1235-06b0-0cc47a392728,"[hi, thanks, i'm, great, friday, right, [noise]]",2017-08-23 14:31:35 UTC
4,002041b8-f3fb-1235-06b0-0cc47a392728,"[tuesday, will, figure, out, a, time, okay, ma...",2017-08-10 21:31:30 UTC
5,002a33b2-f894-1235-06b0-0cc47a392728,"[talk, on, monday, at, one, thirty, i'm, looki...",2017-08-14 18:37:27 UTC
6,00302ae1-f3ed-1235-06b0-0cc47a392728,"[with, that, being, a, friday, [noise], [noise...",2017-08-07 20:13:13 UTC
7,00303564-fc8f-1235-06b0-0cc47a392728,"[and, then, yeah, next, tuesday, i, know, you,...",2017-08-22 19:02:08 UTC
8,00349fba-fd60-1235-06b0-0cc47a392728,"[back, out, wednesday, after, two, [noise], [n...",2017-08-24 18:14:02 UTC
9,0037ecba-dc51-1235-06b0-0cc47a392728,"[i'll, probably, around, like, eleven, tomorro...",2017-07-06 20:15:08 UTC


In [10]:
sentences = final_df['text window']


model = gensim.models.Word2Vec(sentences)
model

2017-10-18 15:20:10,367 : INFO : collecting all words and their counts
2017-10-18 15:20:10,369 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2017-10-18 15:20:10,393 : INFO : PROGRESS: at sentence #10000, processed 92052 words, keeping 2016 word types
2017-10-18 15:20:10,402 : INFO : collected 2331 word types from a corpus of 122050 raw words and 13261 sentences
2017-10-18 15:20:10,403 : INFO : Loading a fresh vocabulary
2017-10-18 15:20:10,406 : INFO : min_count=5 retains 681 unique words (29% of original 2331, drops 1650)
2017-10-18 15:20:10,407 : INFO : min_count=5 leaves 119597 word corpus (97% of original 122050, drops 2453)
2017-10-18 15:20:10,410 : INFO : deleting the raw counts dictionary of 2331 items
2017-10-18 15:20:10,410 : INFO : sample=0.001 downsamples 81 most-common words
2017-10-18 15:20:10,411 : INFO : downsampling leaves estimated 63303 word corpus (52.9% of prior 119597)
2017-10-18 15:20:10,412 : INFO : estimated required memory for 681 w